In [1]:
import pandas as pd
import numpy as np
from os.path import join as JoinPath
from enum import Enum
from typing import List
from copy import deepcopy

### Ładowanie danych

In [2]:
# Enum do indentyfikacji pól danych
class DataID(Enum):
    Electricity_Household_From = 0
    Electricity_Household_Until = 1
    Electricity_Industrial_From = 2
    Electricity_Industrial_Until = 3
    Gas_Household_From = 4
    Gas_Household_Until = 5
    Gas_Industrial_From = 6
    Gas_Industrial_Until = 7

# Kolejność i liczba plików musi być indentyczna co w DataID enum
TsvList = [\
    "Electricity prices for household consumers from 2007.tsv",\
    "Electricity prices for household consumers until 2007.tsv",\
    "Electricity prices for non-household consumers from 2007.tsv",\
    "Electricity prices for non-household consumers until 2007.tsv",\
    "Gas prices for household consumers from 2007.tsv",\
    "Gas prices for household consumers until 2007.tsv",\
    "Gas prices for non-household consumers from 2007.tsv",\
    "Gas prices for non-household consumers until 2007.tsv",\
    ]

In [3]:
# Ładowanie danych
DataFolder = "data-for-stan-models"
LoadedData = {}

for i in range(len(TsvList)): # Iteracja po ładowanych plikach
    LoadedData[i] = pd.read_csv(\
        JoinPath(DataFolder, TsvList[i]),\
        sep='\t',\
        header=0,\
        )
    LoadedData[i].rename(columns = {'freq,product,consom,unit,tax,currency,geo\TIME_PERIOD': 'country'}, inplace=True)
LoadedData[0].head()

,country,2007-S1,2007-S2,2008-S1,2008-S2,2009-S1,2009-S2,2010-S1,2010-S2,2011-S1,...,2017-S1,2017-S2,2018-S1,2018-S2,2019-S1,2019-S2,2020-S1,2020-S2,2021-S1,2021-S2
0,"S,6000,4161901,KWH,I_TAX,EUR,AL",:,:,:,:,:,:,:,:,0.1152,...,0.0844,0.0856,:,0.0910,0.0920 e,0.0933,0.0922 e,0.0920 e,:,0.0937
1,"S,6000,4161901,KWH,I_TAX,EUR,AT",:,0.2653,0.2650,0.2649,0.2727,0.2726,0.2538,0.2570,0.2946,...,0.3699,0.3730,0.3652,0.3573,0.3796,0.3847,0.4074,0.4037,0.4387,0.4545
2,"S,6000,4161901,KWH,I_TAX,EUR,BA",:,:,:,:,:,:,:,:,:,...,0.2224,0.2020,0.2206,0.1884,0.2090,0.1872,0.2130,0.2077,0.2146,0.1876
3,"S,6000,4161901,KWH,I_TAX,EUR,BE",:,0.2443,0.2785,0.3593,0.2628,0.2780,0.2930,0.2852,0.3034,...,0.4988,0.5160,0.4300,0.4358,0.4742,0.5012,0.4393,0.4792,0.4346,0.4078
4,"S,6000,4161901,KWH,I_TAX,EUR,BG",:,0.0741,0.0741,0.0823,0.0844,0.0823,0.0823,0.0842,0.0840,...,0.0977,0.0998,0.1000,0.1016,0.1014,0.1017,0.1034,0.1044,0.1061,0.1119


### Wybieranie danych i zamiana nazewnictwa

In [4]:
def Filter(index: str, unit:str = "KWH", currency: str = "EUR", tax: str = "X_TAX", consom: str = ""):
    '''
    Funkcja sprawdzająca czy to jest wiersz danych który chcemy załadować
    Parameters:
        index (str): Przetwarzany indeks danych
        unit (str): Jednostka danych. Dostępne: "KWH"(Kilo Wato Godzina), "GJ_GCV"(Gigadżul)
        currency (str): Waluta danych. Dostępne: "EUR"(Euro), "PPS"(Purchasing Power Standard), "NAC"(Narodowa waluta)
        tax (str): Jaki podatek. Dostępne: "I_TAX"(wszystkie podatki), "X_VAT"(bez VAT), "X_TAX"(bez podatków)
        consom (str): Jakaś gówniana wartość duplikująca dane XD
    Returns:
        (bool): Czy załadować wiersz
    '''
    # Filtracja parametru
    if unit not in index:
        return False
    if currency not in index:
        return False
    if tax not in index:
        return False
    if consom not in index:
        return False
    # Filtracja niechcianych danych statystycznych o UE
    if "EU15" in index or "EU25" in index or "EU27_2007" in index or "EU27_2020" in index:
        return False
    return True

def FilterRange(data, fun, **kwargs):
    result = []
    for i in data:
        result.append(fun(i, **kwargs))
    return result

def FindConsom(index: str):
    '''
    Funkcja znajduje wartość consom
    '''
    return index.split(',')[2]

In [5]:
# Uwaga funkcja brzydka zależy od danych
FilteredData = {}
for i in range(len(LoadedData.keys())):
    mask = []
    consom = FindConsom(LoadedData[i]['country'][0])
    if(i < 4):
        mask = (FilterRange(LoadedData.get(i)["country"], Filter, unit = "KWH", consom=consom))
    else:
        mask = (FilterRange(LoadedData.get(i)["country"], Filter, unit = "GJ_GCV", consom=consom))
    FilteredData[i] = LoadedData[i][mask]
    print(len(FilteredData[i])) # Wypisanie ilości otrzymanych danych
FilteredData[0].head()

42
31
42
31
34
26
34
26


,country,2007-S1,2007-S2,2008-S1,2008-S2,2009-S1,2009-S2,2010-S1,2010-S2,2011-S1,...,2017-S1,2017-S2,2018-S1,2018-S2,2019-S1,2019-S2,2020-S1,2020-S2,2021-S1,2021-S2
123,"S,6000,4161901,KWH,X_TAX,EUR,AL",:,:,:,:,:,:,:,:,0.0960,...,0.0703,0.0713,:,0.0759,0.0767 e,0.0778,0.0768 e,0.0767 e,:,0.0781
124,"S,6000,4161901,KWH,X_TAX,EUR,AT",:,0.1834,0.1812,0.1812,0.1874,0.1874,0.1746,0.1772,0.2055,...,0.2134,0.2061,0.2105,0.2098,0.2259,0.2304,0.2413,0.2376,0.2500,0.2640
125,"S,6000,4161901,KWH,X_TAX,EUR,BA",:,:,:,:,:,:,:,:,:,...,0.1887,0.1712,0.1870,0.1594,0.1769,0.1580,0.1798,0.1751,0.1808,0.1576
126,"S,6000,4161901,KWH,X_TAX,EUR,BE",:,0.1873,0.2153,0.2809,0.2024,0.2134,0.2241,0.2176,0.2304,...,0.2916,0.2785,0.3013,0.3057,0.3411,0.3683,0.3150,0.3478,0.3080,0.2878
127,"S,6000,4161901,KWH,X_TAX,EUR,BG",:,0.0619,0.0619,0.0685,0.0706,0.0685,0.0685,0.0701,0.0699,...,0.0814,0.0831,0.0833,0.0847,0.0845,0.0848,0.0862,0.0870,0.0885,0.0933


In [6]:
def CreateNewIndex(index: List[str]):
    """
    Funkcja tworzy indeks z identyfikatorem kraju zamiast oryginalnego gówna
    """
    result = []
    for i in index:
        result.append(i[i.rfind(',') + 1:])
    return result

# Filtracja danych
ResultData = {}
for i in range(len(FilteredData.keys())):
    ResultData[i] = FilteredData[i].apply(lambda x: CreateNewIndex(x) if x.name == 'country' else x)
ResultData[0].head()

,country,2007-S1,2007-S2,2008-S1,2008-S2,2009-S1,2009-S2,2010-S1,2010-S2,2011-S1,...,2017-S1,2017-S2,2018-S1,2018-S2,2019-S1,2019-S2,2020-S1,2020-S2,2021-S1,2021-S2
123,AL,:,:,:,:,:,:,:,:,0.0960,...,0.0703,0.0713,:,0.0759,0.0767 e,0.0778,0.0768 e,0.0767 e,:,0.0781
124,AT,:,0.1834,0.1812,0.1812,0.1874,0.1874,0.1746,0.1772,0.2055,...,0.2134,0.2061,0.2105,0.2098,0.2259,0.2304,0.2413,0.2376,0.2500,0.2640
125,BA,:,:,:,:,:,:,:,:,:,...,0.1887,0.1712,0.1870,0.1594,0.1769,0.1580,0.1798,0.1751,0.1808,0.1576
126,BE,:,0.1873,0.2153,0.2809,0.2024,0.2134,0.2241,0.2176,0.2304,...,0.2916,0.2785,0.3013,0.3057,0.3411,0.3683,0.3150,0.3478,0.3080,0.2878
127,BG,:,0.0619,0.0619,0.0685,0.0706,0.0685,0.0685,0.0701,0.0699,...,0.0814,0.0831,0.0833,0.0847,0.0845,0.0848,0.0862,0.0870,0.0885,0.0933


### Łączenie danych

In [7]:
# Łączenie danych
electricity_household = ResultData[1].set_index('country')\
    .join(deepcopy(ResultData[0]).drop(['2007-S1 ', '2007-S2 '], axis=1)\
        .set_index('country'), on="country")
electricity_household.head()

electricity_industry = ResultData[3].set_index('country')\
    .join(deepcopy(ResultData[2]).drop(['2007-S1 ', '2007-S2 '], axis=1)\
        .set_index('country'), on="country")
electricity_industry.head()

gas_household = ResultData[5].set_index('country')\
    .join(deepcopy(ResultData[4]).drop(['2007-S1 ', '2007-S2 '], axis=1)\
        .set_index('country'), on="country")
gas_household.head()

gas_industry = ResultData[7].set_index('country')\
    .join(deepcopy(ResultData[6]).drop(['2007-S1 ', '2007-S2 '], axis=1)\
        .set_index('country'), on="country")
gas_industry.head()

,1985-S1,1986-S1,1987-S1,1988-S1,1989-S1,1990-S1,1991-S1,1991-S2,1992-S1,1992-S2,...,2017-S1,2017-S2,2018-S1,2018-S2,2019-S1,2019-S2,2020-S1,2020-S2,2021-S1,2021-S2
country,,,,,,,,,,,,,,,,,,,,,
AT,:,:,:,:,:,:,:,:,:,:,...,11.9000,12.4317,12.0167,12.1694,11.7750,12.1750,11.5444,11.2417,11.7139,13.3661
BE,8.2752,7.8903,5.5516,5.3835,5.0466,5.7753,6.2446,6.1624,6.0329,5.7803,...,10.6781,11.0109,10.2030,11.5064,11.2987,10.4810,9.7883,9.9763,9.8782,14.1133
BG,:,:,:,:,:,:,:,:,:,:,...,7.0917,8.2531,8.3462,9.6696,9.9266,9.7056,8.9294,7.8782,8.1046,16.9628
CZ,:,:,:,:,:,:,:,:,:,:,...,9.0481,9.7959,9.2951,9.1080,9.3503,9.5618,9.0020,8.7630,8.0237,8.6996
DE,7.6610,7.8524,5.9917,5.3451,4.5103,5.6982,6.3598,6.1869,6.1382,5.6844,...,10.9700,10.7580,10.5630,10.5910,10.4340,10.0422,10.6987,10.6166,10.2412,12.2731


### Znajdywanie części wspólnej i zapis danych

In [8]:
def PrintCount(*args: List):
    '''
    Funkcja wypisuje liczność podanych zbiorów danych
    Params:
        *args (List): Listy do wypisania liczności
    '''
    ElemCount = []
    for i in args:
        ElemCount.append(len(i))
    print('Liczność przeszukiwanych zbiorów:', ElemCount)

def FindIntersection(*args: List):
    '''
    Funkcja znajduje część wspólną wielu zbiorów/list
    Params:
        *args (List): Listy do znalezienia części wspólnej
    Returns:
        (List): Część wspólna
    '''
    result = set(args[0])
    if len(args) > 1:
        for i in range(1, len(args)):
            result = result.intersection(args[i])
    
    print('Ilość wspólnych krajów', len(result))
    print('Wspólne kraje', result)
    return result

def FindDifference(*args):
    '''
    Funkcja znajduje różnicę wielu zbiorów/list
    Params:
        *args (List): Listy do znalezienia części wspulnej
    Returns:
        (List): Różnica
    '''
    result = set([])
    intersection = set(FindIntersection(*args))
    for i in args:
        result.update(intersection.symmetric_difference(i))
    print('Ilość różnych krajów', len(result))
    print('Różne kraje', result)
    return result


In [9]:
# Znajdywanie krajów niewystępujących gdzie indziej i części wspulnej
print('Początkowa liczność')
PrintCount(electricity_household.index,\
    electricity_industry.index,\
    gas_household.index,\
    gas_industry.index)

intersection_keys = FindIntersection(electricity_household.index,\
    electricity_industry.index,\
    gas_household.index,\
    gas_industry.index)

FindDifference(electricity_household.index,\
    electricity_industry.index,\
    gas_household.index,\
    gas_industry.index)

# Przycinanie danych
electricity_household = electricity_household.filter(items = intersection_keys, axis=0)
electricity_industry = electricity_industry.filter(items = intersection_keys, axis=0)
gas_household = gas_household.filter(items = intersection_keys, axis=0)
gas_industry = gas_industry.filter(items = intersection_keys, axis=0)

print('Końcowa liczność')
PrintCount(electricity_household.index,\
    electricity_industry.index,\
    gas_household.index,\
    gas_industry.index)
print('Head pierwszego: electricity_household')
electricity_household.head()


Początkowa liczność
Liczność przeszukiwanych zbiorów: [31, 31, 26, 26]
Ilość wspólnych krajów 25
Wspólne kraje {'TR', 'IT', 'FR', 'SK', 'BE', 'DK', 'SE', 'EA', 'LV', 'RO', 'CZ', 'NL', 'LT', 'BG', 'HU', 'PT', 'ES', 'DE', 'LU', 'AT', 'IE', 'UK', 'PL', 'HR', 'EE'}
Ilość wspólnych krajów 25
Wspólne kraje {'TR', 'IT', 'FR', 'SK', 'BE', 'DK', 'SE', 'EA', 'LV', 'RO', 'CZ', 'NL', 'LT', 'BG', 'HU', 'PT', 'ES', 'DE', 'LU', 'AT', 'IE', 'UK', 'PL', 'HR', 'EE'}
Ilość różnych krajów 6
Różne kraje {'FI', 'MT', 'SI', 'EL', 'CY', 'NO'}
Końcowa liczność
Liczność przeszukiwanych zbiorów: [25, 25, 25, 25]
Head pierwszego: electricity_household


,1985-S1,1986-S1,1987-S1,1988-S1,1989-S1,1990-S1,1991-S1,1991-S2,1992-S1,1992-S2,...,2017-S1,2017-S2,2018-S1,2018-S2,2019-S1,2019-S2,2020-S1,2020-S2,2021-S1,2021-S2
country,,,,,,,,,,,,,,,,,,,,,
TR,:,:,:,:,:,:,:,:,:,:,...,0.0844,0.0772,0.0737,0.0689,0.0683,0.0843,0.0804,0.0659,0.0675,0.0642
IT,:,:,:,:,:,:,:,:,:,:,...,0.2572,0.2217,0.2160,0.2382,0.3494,0.2926,0.3499,0.2877,0.3575,0.3322
FR,:,:,:,:,:,:,:,:,:,:,...,0.2233,0.2114,0.2712,0.3128,0.2497,0.2775,0.2981,0.3247,0.2884,0.3161
SK,:,:,:,:,:,:,:,:,:,:,...,0.1636,0.1595,0.1687,0.1606,0.1778,0.1720,0.1821,0.1902,0.1812,0.1809
BE,0.1700,0.1761,0.1738,0.1760,0.1757,0.1825,0.1836,0.1856,0.1869,0.1892,...,0.2916,0.2785,0.3013,0.3057,0.3411,0.3683,0.3150,0.3478,0.3080,0.2878


In [10]:
empty = electricity_household.iloc[1][1]
print(type(empty), '\'', empty, '\'')
# electricity_household.applymap(lambda x: np.nan if x==empty else x)

<class 'str'> ' :  '


In [11]:
# empty = electricity_household.iloc[1][1]
# print(type(empty), '\'', empty, '\'')
def RefactorData(df):
    df = df.replace(' :  ', np.NaN)
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            elem = df.iloc[i][j]
            if isinstance(elem, str):
                newElem = ''
                for k in range(len(elem)):
                    if elem[k] in '1234567890.':
                        newElem += elem[k]
                df.iloc[i][j] = newElem
    return df

In [12]:
# # Zapisywanie danych
DataFrameToSave = electricity_household
DataFrameToSave = RefactorData(DataFrameToSave)
DataFrameToSaveName = "electricity_household"
print("Ilość krajów:", len(DataFrameToSave), ", zmienna:", DataFrameToSaveName)
DataFrameToSave.to_csv(DataFrameToSaveName + '.tsv', index=True, sep='\t')

DataFrameToSave = electricity_industry
DataFrameToSave = RefactorData(DataFrameToSave)
DataFrameToSaveName = "electricity_industry"
print("Ilość krajów:", len(DataFrameToSave), ", zmienna:", DataFrameToSaveName)
DataFrameToSave.to_csv(DataFrameToSaveName + '.tsv', index=True, sep='\t')

DataFrameToSave = gas_household
DataFrameToSave = RefactorData(DataFrameToSave)
DataFrameToSaveName = "gas_household"
print("Ilość krajów:", len(DataFrameToSave), ", zmienna:", DataFrameToSaveName)
DataFrameToSave.to_csv(DataFrameToSaveName + '.tsv', index=True, sep='\t')

DataFrameToSave = gas_industry
DataFrameToSave = RefactorData(DataFrameToSave)
DataFrameToSaveName = "gas_industry"
print("Ilość krajów:", len(DataFrameToSave), ", zmienna:", DataFrameToSaveName)
DataFrameToSave.to_csv(DataFrameToSaveName + '.tsv', index=True, sep='\t')

Ilość krajów: 25 , zmienna: electricity_household
Ilość krajów: 25 , zmienna: electricity_industry
Ilość krajów: 25 , zmienna: gas_household
Ilość krajów: 25 , zmienna: gas_industry


In [13]:
# RefactorData(electricity_household)
electricity_household.dtypes

1985-S1     object
1986-S1     object
1987-S1     object
1988-S1     object
1989-S1     object
             ...  
2019-S2     object
2020-S1     object
2020-S2     object
2021-S1     object
2021-S2     object
Length: 68, dtype: object